### Prédiction de la valeur des rendements

Dans une seconde approche, nous cherchons à prédire directement la valeur du rendement futur, ce qui conduit naturellement à formuler le problème comme un problème de régression. Contrairement à la classification directionnelle, l’objectif ici est d’estimer l’amplitude du rendement, et non seulement son signe.

Cette formulation est plus exigeante, mais aussi plus informative d’un point de vue économique. Prédire la valeur du rendement permet, en principe, de construire des stratégies plus fines, telles que des allocations proportionnelles aux rendements attendus, des classements cross-sectionnels des actifs, ou encore des stratégies long/short pondérées par l’intensité du signal.

Cependant, il est crucial de souligner que la prédiction de la valeur des rendements constitue un problème intrinsèquement complexe, en raison de la nature même des marchés financiers et de l’information dont nous disposons. Les rendements dépendent d’une multitude de facteurs exogènes, souvent imprévisibles et en grande partie absents de notre jeu de données : événements géopolitiques, décisions de politique monétaire, annonces macroéconomiques inattendues, chocs météorologiques affectant certaines industries, évolutions réglementaires, ou encore dynamiques comportementales des investisseurs. Une part substantielle des variations de prix est ainsi liée à de l’information nouvelle, par définition non observable au moment de la prédiction.

De plus, même parmi les facteurs observables, notre dataset ne capture qu’un sous-ensemble très restreint de l’information pertinente. Les variables utilisées (prix passés, volumes, indicateurs techniques) ne reflètent qu’indirectement l’ensemble des anticipations, contraintes et stratégies d’une population d’agents hétérogènes. Les marchés agrègent en permanence ces informations dispersées, ce qui rend les relations entre variables explicatives et rendements futurs à la fois faibles, non linéaires et instables dans le temps.

Dans ce contexte, il est important de souligner que même une capacité prédictive extrêmement limitée constitue déjà une prouesse. Lorsqu’un modèle parvient à extraire un signal, aussi faible soit-il, dans un environnement dominé par l’incertitude et l’information non observable, cela signifie qu’il capte une régularité exploitable au-delà du bruit. En finance, une amélioration marginale de la prévision — même de très faible amplitude — peut suffire à générer une espérance de gain positive lorsqu’elle est correctement exploitée, par exemple via des stratégies long/short, des classements relatifs ou des agrégations de signaux à grande échelle. De telles stratégies, lorsqu’elles sont robustes et appliquées de manière systématique, peuvent produire des performances économiquement significatives et représenter une source de valeur substantielle, malgré des métriques statistiques (comme le R²) apparemment modestes.

C’est néanmoins cette formulation qui est la plus couramment adoptée dans la littérature empirique récente sur la prédiction des rendements, notamment dans les travaux utilisant des méthodes de machine learning. Des articles de référence, comme Gu, Kelly & Xiu (2020), évaluent explicitement la capacité des modèles à prédire les rendements excédentaires en niveau et utilisent des métriques telles que la MSE ou le R² hors-échantillon pour quantifier le pouvoir prédictif.

Cette seconde tentative vise donc à répondre à la question : dans quelle mesure les variables considérées permettent-elles de prédire l’intensité des rendements futurs, au-delà de leur simple direction, et comment ce pouvoir prédictif — nécessairement limité — se compare-t-il aux ordres de grandeur observés dans la littérature.


In [283]:
import numpy as np
import pandas as pd

In [284]:
df = pd.read_csv("../sp500_ohlcv_2005_2025_2.csv")
sp500 = pd.read_csv("../sectors.csv")

df = df.merge(
    sp500[['Ticker', 'Sector']], 
    on='Ticker', 
    how='left'
)

df.head()

,Date,Ticker,Open,High,Low,Close,Volume,Sector
0,2005-01-03,MMM,37.356306,37.915495,37.301752,37.460873,3817632.0,Industrials
1,2005-01-04,MMM,37.460883,37.742749,37.129005,37.156284,4358942.0,Industrials
2,2005-01-05,MMM,37.142662,37.256318,36.701679,36.701679,3462779.0,Industrials
3,2005-01-06,MMM,36.769829,37.460855,36.742550,37.033508,3605342.0,Industrials
4,2005-01-07,MMM,37.051714,37.642720,36.938058,37.415409,3938428.0,Industrials


In [285]:

# Suppose df a les colonnes : Date, Ticker, Open, High, Low, Close, Volume
df = df.sort_values(["Ticker", "Date"])

# --- 1) Log-return
df["log_return"] = np.log(df["Close"]) - np.log(df["Close"].shift(1))

# --- 2) Target = log-return de demain (r_{t+1})
df["target"] = df.groupby("Ticker")["log_return"].shift(-1)

# --- 3) Features momentum
df["mom_5"]  = df.groupby("Ticker")["Close"].transform(lambda x: x / x.shift(5) - 1)
df["mom_21"] = df.groupby("Ticker")["Close"].transform(lambda x: x / x.shift(21) - 1)

# --- 4) Features volatilité
df["vol_5"]  = df.groupby("Ticker")["log_return"].transform(lambda x: x.rolling(5).std())
df["vol_21"] = df.groupby("Ticker")["log_return"].transform(lambda x: x.rolling(21).std())

# --- 5) High–Low range
df["range"] = (df["High"] - df["Low"]) / df["Open"]

# --- 6) Volume z-score
df["volume_z"] = df.groupby("Ticker")["Volume"].transform(
    lambda x: (x - x.mean()) / x.std()
)

df['Return'] = df.groupby('Ticker')['Close'].pct_change(fill_method=None)

# --- 7) Clean
df = df.dropna()


In [286]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


def regression(df, features):
    """
    Entraîne une régression linéaire sur un sous-échantillon temporel (2010–2021),
    avec un split train/val/test réalisé *par dates* (toutes les lignes d'une même
    date vont dans le même split), puis évalue sur le test et calcule un RMSE par secteur.

    Paramètres
    ----------
    df : pd.DataFrame
        Doit contenir au minimum : "Date", "target", "Sector" + les colonnes de `features`.
        Chaque ligne correspond typiquement à (Date, action) avec des features associées.
    features : list[str]
        Liste des noms de colonnes à utiliser comme variables explicatives.

    Retours
    -------
    model : LinearRegression
        Modèle sklearn entraîné.
    sector_rmse : pd.Series
        RMSE par secteur (index = secteurs), trié du meilleur au pire.
    """

    # -------------------------
    # 1) Restriction temporelle
    # -------------------------
    # On garde uniquement les observations entre 2010-01-01 et 2021-12-31 (inclus).
    # NB : si df["Date"] est déjà un datetime, c'est parfait. Sinon, la comparaison
    # de chaînes peut produire des effets inattendus → à convertir en amont idéalement.
    mask = (df["Date"] >= "2010-01-01") & (df["Date"] <= "2021-12-31")
    df_period = df.loc[mask].copy()

    # ---------------------------------------
    # 2) Split train/val/test (par *dates*)
    # ---------------------------------------
    # Objectif : éviter de mettre la même date à la fois dans train et test.
    # On récupère la liste des dates uniques, puis on la mélange aléatoirement.
    unique_dates = df_period["Date"].unique()

    rng = np.random.default_rng(seed=42)  # seed fixé → résultats reproductibles
    rng.shuffle(unique_dates)             # mélange des dates (attention: non chronologique)

    # Proportions : 60% train, 20% val, 20% test (approx).
    n_dates = len(unique_dates)
    n_train = int(0.6 * n_dates)
    n_val   = int(0.2 * n_dates)
    # le reste est attribué au test

    train_dates = unique_dates[:n_train]
    val_dates   = unique_dates[n_train:n_train + n_val]
    test_dates  = unique_dates[n_train + n_val:]

    # On sélectionne les lignes correspondant aux dates de chaque split
    train = df_period[df_period["Date"].isin(train_dates)]
    val   = df_period[df_period["Date"].isin(val_dates)]
    test  = df_period[df_period["Date"].isin(test_dates)].copy()  # copy car on ajoute une colonne ensuite

    # ---------------------------------------
    # 3) Préparation des matrices X / y
    # ---------------------------------------
    X_train, y_train = train[features], train["target"]
    X_val, y_val     = val[features],   val["target"]   # actuellement non utilisé (mais prêt pour tuning)
    X_test, y_test   = test[features],  test["target"]

    # -------------------------
    # 4) Entraînement du modèle
    # -------------------------
    # LinearRegression = OLS (avec intercept) sans régularisation.
    model = LinearRegression()
    model.fit(X_train, y_train)

    # -------------------------
    # 5) Prédiction et métriques
    # -------------------------
    y_pred = model.predict(X_test)

    # Métriques globales de régression
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae  = mean_absolute_error(y_test, y_pred)
    r2   = r2_score(y_test, y_pred)

    # Variance de la cible sur le test (utile pour comparer l'échelle des erreurs)
    var_test = np.var(y_test)

    print("RMSE :", rmse)
    print("MAE  :", mae)
    print("R²   :", r2)
    print("Var(y_test) :", var_test)

    # ---------------------------------------
    # 6) RMSE par secteur (diagnostic)
    # ---------------------------------------
    # On ajoute la prédiction au DataFrame de test
    test["pred"] = y_pred

    # Pour chaque secteur, on calcule le RMSE sur les lignes de ce secteur
    sector_rmse = (
        test.groupby("Sector")
            .apply(
                lambda g: np.sqrt(mean_squared_error(g["target"], g["pred"])),
                include_groups=False
            )
            .sort_values()
    )

    return model, sector_rmse


In [287]:
regression(df, ["mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"])


RMSE : 0.020804340409584315
MAE  : 0.012958511181494624
R²   : -0.007582188713726623
Var(y_test) : 0.0004295635479924462


(LinearRegression(),
 Sector
 Consumer Staples          0.015680
 Utilities                 0.016088
 Real Estate               0.019053
 Health Care               0.019305
 Financials                0.019404
 Industrials               0.020485
 Materials                 0.021868
 Information Technology    0.023075
 Communication Services    0.023257
 Consumer Discretionary    0.023942
 Energy                    0.027032
 dtype: float64)

### Interprétation des résultats du modèle de régression linéaire

Le modèle de régression linéaire appliqué à notre cible produit les performances suivantes :

- RMSE : 0.02078  
- MAE : 0.01294  
- R² : –0.0076

Plusieurs éléments se dégagent immédiatement :

1. Le R² négatif indique que la régression linéaire fait moins bien qu’un modèle trivial qui prédirait simplement la moyenne historique de la variable cible. Autrement dit, la variance expliquée est nulle, voire légèrement détériorée.

2. Le RMSE et le MAE sont proches de la dispersion naturelle du rendement à prédire, ce qui montre que le modèle n’arrive pas à extraire un signal utile à partir des caractéristiques disponibles.

3. Ce résultat est cohérent avec la littérature : les rendements financiers sont notoirement difficiles à modéliser de manière linéaire, en raison de relations potentiellement non linéaires, interactives, et faiblement signalées.

Ces observations suggèrent que la régression linéaire est trop restrictive pour capturer les relations complexes entre les variables explicatives et les rendements futurs.  
Pour cette raison, nous passons à un modèle plus flexible — XGBoost — qui permet de modéliser des non-linéarités, des interactions entre features et une structure plus riche. Nous espérons ainsi obtenir une amélioration significative en termes d’erreur de prédiction et de R² hors-échantillon.


In [288]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


def regression_xgb(df, features):
    """
    Entraîne un modèle XGBoost de régression (XGBRegressor) pour prédire `target`
    à partir de `features`, avec un split temporel :

      - train : dates < 2018-01-01
      - val   : 2018-01-01 <= dates < 2021-01-01
      - test  : dates >= 2021-01-01

    On utilise `val` pour l'early stopping, puis on évalue sur `test`.
    Enfin, on calcule une métrique par secteur.

    Paramètres
    ----------
    df : pd.DataFrame
        Doit contenir : "Date", "target", "Sector" + les colonnes de `features`.
    features : list[str]
        Colonnes utilisées comme variables explicatives.

    Retours
    -------
    sector_metric : pd.Series
        Série indexée par "Sector", triée par valeur croissante.
    """

    # Copie défensive pour éviter de modifier le df original
    df = df.copy()

    # ----------------------------------------
    # 1) Split train/val/test (chronologique)
    # ----------------------------------------
    # Objectif : mimer un vrai cadre out-of-sample (train sur le passé, test sur le futur).
    train = df[df["Date"] < "2018-01-01"]
    val   = df[(df["Date"] >= "2018-01-01") & (df["Date"] < "2021-01-01")]
    test  = df[df["Date"] >= "2021-01-01"].copy()  # copy car on ajoute ensuite une colonne "pred"

    X_train, y_train = train[features], train["target"]
    X_val,   y_val   = val[features],   val["target"]
    X_test,  y_test  = test[features],  test["target"]

    # -------------------------
    # 2) Modèle XGBoost
    # -------------------------
    # Hyperparamètres "raisonnables" pour un baseline :
    # - n_estimators : nombre max d'arbres (l'early stopping peut arrêter avant)
    # - max_depth    : complexité de chaque arbre
    # - learning_rate: shrinkage (plus petit = plus stable mais nécessite + d'arbres)
    # - subsample / colsample_bytree : bagging de lignes/colonnes (régularisation)
    # - objective    : perte de régression MSE
    # - early_stopping_rounds : stop si la perf sur `val` n'améliore plus pendant 30 itérations
    model = XGBRegressor(
        n_estimators=300,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        n_jobs=-1,
        random_state=42,
        early_stopping_rounds=30,
    )

    # -------------------------
    # 3) Entraînement + early stopping
    # -------------------------
    # eval_set : on fournit la validation pour surveiller la perf.
    # verbose=False : silence pendant l'entraînement.
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # -------------------------
    # 4) Prédiction sur le test
    # -------------------------
    y_pred = model.predict(X_test)

    # -------------------------
    # 5) Métriques globales (test)
    # -------------------------
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae  = mean_absolute_error(y_test, y_pred)
    r2   = r2_score(y_test, y_pred)

    print("RMSE :", rmse)
    print("MAE  :", mae)
    print("R²   :", r2)

    # On stocke les prédictions pour pouvoir grouper et diagnostiquer
    test["pred"] = y_pred

    # -------------------------
    # 6) Métrique par secteur
    sector_r2 = (
        test.groupby("Sector")
            .apply(lambda g: r2_score(g["target"], g["pred"]), include_groups=False)
            .sort_values()
    )

    return sector_r2


In [289]:
regression_xgb(df, ["Open", "Close", "High", "Low", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"])

RMSE : 0.02018808612331705
MAE  : 0.013768098419402148
R²   : 4.016453471267223e-05


Sector
Consumer Staples         -0.000668
Energy                   -0.000592
Health Care              -0.000386
Communication Services   -0.000157
Materials                -0.000060
Utilities                -0.000014
Real Estate               0.000008
Consumer Discretionary    0.000040
Industrials               0.000047
Financials                0.000058
Information Technology    0.000125
dtype: float64

### Limites du découpage temporel strict et motivation d’une méthodologie alternative

Dans un premier temps, nous avons évalué le modèle selon un protocole standard en finance empirique, en l’entraînant sur une première plage temporelle (données antérieures à 2018) et en l’évaluant hors-échantillon sur une période ultérieure (données postérieures à 2018). Ce découpage, conforme à la logique de prévision du futur à partir du passé, vise à reproduire les conditions réelles d'utilisation d'un tel modèle par un fond d'investissement. Toutefois, dans ce cadre, le modèle obtient un R² hors-échantillon quasiement nul, indiquant qu’il ne parvient pas à améliorer la prédiction par rapport à une simple constante (la moyenne de la cible).

Ce résultat suggère que, sur un bloc temporel futur strictement séparé, les relations estimées dans le passé ne sont pas suffisamment stables pour fournir un pouvoir prédictif exploitable. Cela peut s’expliquer par plusieurs facteurs : changements de régime de marché, instabilité des relations entre caractéristiques et rendements, ou encore un rapport signal-bruit particulièrement faible à court horizon.

Afin d’aller au-delà de ce constat et d’évaluer si les variables considérées contiennent néanmoins un signal prédictif exploitable d’un point de vue statistique, nous adoptons alors une méthodologie alternative. L’idée est de répartir plus équitablement les dates de l’ensemble de l’échantillon entre les jeux d’entraînement, de validation et de test, plutôt que de concentrer le test sur un seul bloc temporel terminal. Cette approche se rapproche davantage d’un cadre de prédiction i.i.d., permettant de mieux mesurer la capacité intrinsèque du modèle à capturer une relation entre caractéristiques et rendements.

Toutefois, une telle répartition des dates sur l’ensemble de la période soulève immédiatement un risque majeur de fuite d’information (data leakage) : dans la mesure où la cible est le rendement futur, on pourrait trouver une date dans l'échantillon d'entrainement qui serait une date target dans l'échantillon de test, ie une date pour laquelle on souhaite prédire les rendements (rendements déjà rencontrés dans l'échantillon d'entrainement). Pour pallier ce problème, nous imposons une contrainte supplémentaire : les dates conservées dans tout le dataset doivent être espacées d’au moins 1 périodes, de sorte qu’aucune observation ne puisse apparaître simultanément comme donnée explicative et comme composante de la cible d’une autre observation. Ce sous-échantillonnage temporel permet ainsi de répartir les dates sur l’ensemble du dataset tout en préservant l’indépendance effective entre observations et en évitant toute utilisation implicite d’informations futures.

Cette méthodologie ne vise pas à reproduire fidèlement un cadre de production pour un fonds d’investissement, mais à répondre à une question complémentaire : existe-t-il, indépendamment des contraintes temporelles strictes, un signal prédictif mesurable dans les données lorsque l’on contrôle soigneusement les risques de data leakage ?


In [290]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


def xgb(df, features, H):
    """
    Pipeline XGBoost pour prédire une cible "forward" sur horizon H :

    - target(t, i) = moyenne des log_return de l'actif i sur les H prochains jours
                     (rolling(H).mean().shift(-H))
      => la target utilise des infos futures, donc on doit être très prudent sur le split.

    - target_date = date du dernier jour utilisé pour construire la target (t+H)
      (utile pour raisonner sur l'horizon, même si ici on split sur df["Date"]).

    Ensuite :
    - restriction à 2010–2021
    - split train/val/test par dates *chronologiques*
    - "thinning" des dates utilisées : on ne garde qu'une date toutes les (H+1) dates
      pour réduire l'overlap de fenêtres et limiter le leak mécanique lié au fait que
      des targets consécutives partagent une partie du futur.

    Enfin :
    - entraînement XGBRegressor avec early stopping sur val
    - évaluation sur test (RMSE/MAE/R² + Var(y_test))
    - métrique par secteur (attention : dans le code original c'est du R² par secteur)

    Paramètres
    ----------
    df : pd.DataFrame
        Doit contenir : "Date", "Ticker", "Sector", "log_return" + features.
        Une ligne ~ (Date, Ticker).
    features : list[str]
        Colonnes explicatives.
    H : int
        Horizon en jours (nombre de jours futurs utilisés pour la target).

    Retours
    -------
    sector_r2 : pd.Series
        R² par secteur (trié).
    test_r2 : float
        R² global sur le test.
    """

    # Copie défensive pour ne pas modifier le df original
    df = df.copy()

    # ---------------------------------------------------------
    # 1) Construction de la target forward (horizon H)
    # ---------------------------------------------------------
    # Pour chaque Ticker, on calcule la moyenne glissante sur H jours,
    # puis on la décale de -H pour que la valeur soit alignée sur la date t
    # (target à la date t = moyenne des returns sur [t, ..., t+H-1] selon l'indexation).
    df["target"] = (
        df.groupby("Ticker")["log_return"]
          .transform(lambda s: s.rolling(H).mean().shift(-H))
    )

    # Date associée à la réalisation de la target (dernier jour utilisé) ~ t+H
    df["target_date"] = df.groupby("Ticker")["Date"].shift(-H)

    # Suppression des bords de série (où le futur manque => target NaN)
    df = df.dropna(subset=["target", "target_date"])

    # ---------------------------------------------------------
    # 2) Restriction temporelle globale
    # ---------------------------------------------------------
    mask = (df["Date"] >= "2010-01-01") & (df["Date"] <= "2021-12-31")
    df_period = df.loc[mask].copy()

    # ---------------------------------------------------------
    # 3) Choix des dates utilisées : thinning pour limiter overlap
    # ---------------------------------------------------------
    # On récupère toutes les dates disponibles (triées).
    # Puis on ne garde qu'une date toutes les (H+1) dates.
    # Intuition : deux dates t et t+1 ont des targets très corrélées car elles partagent
    # beaucoup de jours futurs dans la moyenne; ça peut rendre l'évaluation trop optimiste
    # si train/test contiennent des dates proches.
    all_dates = np.sort(df_period["Date"].unique())
    used_dates = all_dates[:: (H + 1)]

    # Split 60/20/20 sur ces dates "décimées", en respectant l'ordre chronologique
    n = len(used_dates)
    i1 = int(0.6 * n)
    i2 = int(0.8 * n)

    train_dates = used_dates[:i1]
    val_dates   = used_dates[i1:i2]
    test_dates  = used_dates[i2:]

    # On forme les ensembles : toutes les lignes dont la Date appartient à chaque bloc
    train = df_period[df_period["Date"].isin(train_dates)]
    val   = df_period[df_period["Date"].isin(val_dates)]
    test  = df_period[df_period["Date"].isin(test_dates)].copy()

    X_train, y_train = train[features], train["target"]
    X_val,   y_val   = val[features],   val["target"]
    X_test,  y_test  = test[features],  test["target"]

    # ---------------------------------------------------------
    # 4) Modèle XGBoost + early stopping
    # ---------------------------------------------------------
    model = XGBRegressor(
        n_estimators=300,
        max_depth=5,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        n_jobs=-1,
        random_state=42,
        early_stopping_rounds=30,
    )

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    # ---------------------------------------------------------
    # 5) Prédictions et métriques globales
    # ---------------------------------------------------------
    y_pred = model.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae  = mean_absolute_error(y_test, y_pred)
    test_r2 = r2_score(y_test, y_pred)
    var_test = np.var(y_test)

    print("RMSE :", rmse)
    print("MAE  :", mae)
    print("R²   :", test_r2)
    print("Var(y_test) :", var_test)

    # Stockage pour diagnostic par groupe
    test["pred"] = y_pred

    # ---------------------------------------------------------
    # 6) Métrique par secteur
    # ---------------------------------------------------------
    sector_r2 = (
        test.groupby("Sector")
            .apply(lambda g: r2_score(g["target"], g["pred"]), include_groups=False)
            .sort_values()
    )

    return sector_r2, test_r2


In [291]:
xgb(df, ["Open", "Close", "High", "Low", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"], 1)

RMSE : 0.025317301950024348
MAE  : 0.016169723488936045
R²   : -1.7216072269121696e-05
Var(y_test) : 0.0006409547433055247


(Sector
 Energy                   -0.001586
 Financials               -0.000844
 Information Technology   -0.000673
 Health Care              -0.000663
 Consumer Discretionary   -0.000593
 Industrials              -0.000561
 Materials                -0.000412
 Real Estate               0.000276
 Utilities                 0.001741
 Communication Services    0.002190
 Consumer Staples          0.004514
 dtype: float64,
 -1.7216072269121696e-05)

Essayons désormais de prédire non plus le rendement à un jour, mais la **moyenne des rendements futurs sur un horizon \(H > 1\)**. Ce changement de cible est motivé par plusieurs considérations à la fois statistiques et économiques.

D’un point de vue statistique, les rendements à très court terme sont fortement dominés par du bruit microstructurel, ce qui limite fortement la capacité prédictive des modèles, même flexibles. En considérant une moyenne de rendements sur plusieurs jours, on procède à une agrégation temporelle qui permet de réduire la variance du bruit et d’améliorer le rapport signal–bruit de la variable cible.

D’un point de vue économique, de nombreux mécanismes plausibles (momentum à court et moyen terme, diffusion progressive de l’information, ajustements sectoriels ou macroéconomiques) s’expriment à des horizons de quelques jours à quelques semaines plutôt qu’à l’échelle journalière. Prédire une moyenne de rendements sur un horizon \(H > 1\) permet donc de mieux capter ces dynamiques sous-jacentes.

Enfin, cette approche est également cohérente avec la littérature empirique, qui montre que la prédictibilité des rendements est souvent plus élevée à des horizons intermédiaires qu’à l’horizon journalier, tandis qu’elle tend à se dégrader à des horizons trop longs. Nous espérons ainsi que le passage à des horizons \(H > 1\) conduira à des performances de prédiction améliorées, mesurées notamment par une baisse des erreurs de prédiction et une augmentation du R² hors-échantillon.


In [292]:
xgb(df, ["Open", "Close", "High", "Low", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"], 5)

RMSE : 0.012168327089261848
MAE  : 0.007607085605767072
R²   : 0.0033383739622092623
Var(y_test) : 0.00014856414683076137


(Sector
 Energy                   -0.017291
 Consumer Staples         -0.009473
 Utilities                -0.002552
 Communication Services   -0.001907
 Real Estate               0.001525
 Materials                 0.003810
 Health Care               0.003963
 Financials                0.004684
 Industrials               0.005460
 Consumer Discretionary    0.010127
 Information Technology    0.011696
 dtype: float64,
 0.0033383739622092623)

In [293]:
xgb(df, [ "Close", "Low", "High", "Open", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"], 21)

RMSE : 0.005167912558057384
MAE  : 0.0033930680111847523
R²   : 0.0028771991372952277
Var(y_test) : 2.678438421488326e-05


(Sector
 Real Estate              -0.006824
 Information Technology   -0.005574
 Health Care              -0.005050
 Consumer Staples         -0.003562
 Consumer Discretionary   -0.001459
 Materials                 0.000620
 Financials                0.003247
 Industrials               0.004053
 Utilities                 0.006082
 Communication Services    0.007454
 Energy                    0.009502
 dtype: float64,
 0.0028771991372952277)

In [294]:
xgb(df, [ "Close", "Low", "High", "Open", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"], 50)

RMSE : 0.003255322651162046
MAE  : 0.002304830580333548
R²   : 0.0026715189053980737
Var(y_test) : 1.0625511818872341e-05


(Sector
 Utilities                -0.060184
 Information Technology   -0.013179
 Real Estate              -0.002790
 Health Care              -0.002003
 Materials                -0.000782
 Consumer Discretionary    0.001195
 Financials                0.001757
 Industrials               0.002952
 Consumer Staples          0.003434
 Energy                    0.004256
 Communication Services    0.004262
 dtype: float64,
 0.0026715189053980737)

### Interprétation des performances par secteur et selon l’horizon \(H\)

Les tableaux ci-dessus présentent les performances du modèle (mesurées par le R² hors-échantillon) par secteur pour différents horizons de prévision \(H\).

Un premier constat général est que le R² augmente lorsque l’horizon de prévision passe de \(H = 1\) à \(H = 5\), puis à \(H = 21\). Cette évolution est cohérente avec la littérature sur la prévision des rendements financiers. À très court horizon (\(H = 1\)), les rendements sont largement dominés par du bruit microstructurel et idiosyncratique (bid–ask bounce, effets de liquidité, chocs transitoires), ce qui rend la prédiction extrêmement difficile. Ce point est bien documenté, notamment dans les travaux de Campbell, Lo et MacKinlay (1997) et de Hasbrouck (2007), qui montrent que la variance des rendements journaliers est en grande partie non prévisible.

Lorsque l’horizon de prévision augmente, la cible correspond à une moyenne de rendements futurs, ce qui a pour effet de lisser le bruit de court terme et de faire émerger des composantes plus persistantes, telles que le momentum ou certaines expositions factorielles. De nombreux travaux empiriques montrent que ces effets sont plus visibles à des horizons de quelques semaines à quelques mois (Jegadeesh et Titman, 1993 ; Moskowitz, Ooi et Pedersen, 2012). Dans ce cadre, il est donc naturel d’observer une amélioration du R² lorsque l’on passe d’un horizon journalier à un horizon intermédiaire.

Cette amélioration est particulièrement marquée pour des secteurs comme l’Information Technology, les Financials ou les Industrials, dont les rendements sont davantage liés à des dynamiques économiques ou financières de moyen terme, plutôt qu’à des fluctuations purement journalières.

En revanche, lorsque l’horizon devient plus long (\(H = 50\)), le R² diminue à nouveau. À ces horizons, plusieurs phénomènes peuvent expliquer la dégradation des performances :
- l’augmentation de \(H\) réduit mécaniquement le nombre d’observations effectives (H observations ignorées sur H+1 pour éviter dataleak), ce qui dégrade le rapport signal/bruit et la précision de l’estimation.
- le lien entre les variables explicatives utilisées (principalement de court à moyen terme) et les rendements très futurs devient plus faible ;
- les rendements à long horizon intègrent des chocs macroéconomiques, réglementaires ou géopolitiques difficilement capturables par les caractéristiques considérées ;

Cette perte de pouvoir prédictif à long horizon est également soulignée dans la littérature sur la prévisibilité de la prime de risque actions (Goyal et Welch, 2008 ; Boudoukh et al., 2008), qui montre que la prévision devient instable et fortement dépendante de la période considérée.

Ainsi, ces résultats suggèrent l’existence d’un horizon intermédiaire optimal (ici autour de quelques semaines) pour lequel le compromis entre réduction du bruit à court terme et perte d’information à long terme est le plus favorable. Ce comportement est conforme à ce que l’on observe classiquement dans la littérature sur la prévision des rendements, où les modèles ont tendance à mieux performer à des horizons intermédiaires qu’à des horizons très courts ou très longs.

Essayons d'améliorer ces résutlats un peu plus en spécialisant un modèlé en l'entrainant sur un seul Ticker

In [295]:
def xgb_ticker(df, ticker, features, H):
    """
    Wrapper autour de `xgb` pour entraîner/évaluer le modèle sur un seul Ticker.

    Idée :
    - On isole d'abord les lignes correspondant à `ticker`.
    - On appelle ensuite la fonction `xgb` (qui construit la target forward, split,
      entraîne XGBoost, calcule les métriques et renvoie les scores).

    Paramètres
    ----------
    df : pd.DataFrame
        DataFrame complet contenant plusieurs tickers.
        Doit contenir au minimum : "Ticker", "Date", "log_return" + features
        (et potentiellement "Sector" si `xgb` calcule des métriques par secteur).
    ticker : str
        Symbole de l'action à sélectionner (ex: "AAPL").
    features : list[str]
        Colonnes explicatives à utiliser.
    H : int
        Horizon en jours pour la définition de la target (moyenne des log_return futurs).

    Retour
    ------
    Même sortie que `xgb(df_sec, features, H)`.

    Remarque importante :
    - Si `xgb` calcule une métrique "par secteur" via groupby("Sector"),
      alors sur un seul ticker :
        * soit tu as exactement 1 seul secteur => la "série par secteur" n'a qu'une valeur,
        * soit la colonne "Sector" n'existe pas / est vide => ça plantera.
      Dans ce cas, il vaut mieux adapter `xgb` pour proposer un mode "ticker-only"
      (par ex. retourner plutôt des métriques globales + éventuellement par date).
    """

    # 1) Filtrage sur le ticker demandé
    # .copy() pour éviter SettingWithCopyWarning et toute modification du df original
    df_ticker = df[df["Ticker"] == ticker].copy()

    # 2) Appel du pipeline principal sur ce sous-ensemble
    return xgb(df_ticker, features, H)


In [296]:
xgb_ticker(df, "HD", [ "Close", "Low", "High", "Open", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"], 1)

RMSE : 0.017018133230228067
MAE  : 0.0115406296138189
R²   : 0.020610443492290464
Var(y_test) : 0.00029571160598700243


(Sector
 Consumer Discretionary    0.02061
 dtype: float64,
 0.020610443492290464)

In [297]:
xgb_ticker(df, "AAPL", [ "Close", "Low", "High", "Open", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"], 1)

RMSE : 0.01889231518191559
MAE  : 0.013517061599754262
R²   : 0.022540170870683718
Var(y_test) : 0.00036515011900874585


(Sector
 Information Technology    0.02254
 dtype: float64,
 0.022540170870683718)

In [298]:
xgb_ticker(df, "PEP", [ "Close", "Low", "High", "Open", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"], 1)

RMSE : 0.01482596986750148
MAE  : 0.009093535629018712
R²   : 0.02370529077288086
Var(y_test) : 0.00022514654687217684


(Sector
 Consumer Staples    0.023705
 dtype: float64,
 0.02370529077288086)

In [299]:
xgb_ticker(df, "PLD", [ "Close", "Low", "High", "Open", "mom_5", "mom_21", "vol_5", "vol_21", "range", "volume_z"], 1)

RMSE : 0.01954177952406527
MAE  : 0.012451713667786374
R²   : 0.027369298213454574
Var(y_test) : 0.00039262707445460084


(Sector
 Real Estate    0.027369
 dtype: float64,
 0.027369298213454574)

### Spécialisation du modèle par titre et interprétation des R² obtenus

Lorsque l’on restreint l’estimation du modèle à un ticker donné, on observe des R² hors-échantillon sensiblement plus élevés, pouvant atteindre des valeurs autour de 0.02–0.03. Ce résultat s’explique par plusieurs mécanismes complémentaires.

Premièrement, la spécialisation par titre permet de réduire fortement l’hétérogénéité structurelle présente dans le panel agrégé. Dans un modèle estimé sur l’ensemble des actions, les relations entre caractéristiques (momentum, volatilité, prix, volumes) et rendements futurs peuvent varier considérablement d’un titre à l’autre en raison de différences de modèle économique, de liquidité, de structure actionnariale ou de sensibilité aux facteurs macroéconomiques. En se concentrant sur un seul ticker, le modèle n’a plus à moyenner ces relations hétérogènes et peut apprendre une dynamique plus stable et spécifique.

Deuxièmement, certains titres présentent des comportements temporels plus persistants que d’autres (par exemple des effets de momentum plus marqués, une volatilité plus prévisible ou des cycles propres liés à leur activité). Dans ces cas, un modèle dédié est naturellement mieux à même de capter ces régularités idiosyncratiques, ce qui se traduit par une amélioration du pouvoir explicatif mesuré par le R².

Troisièmement, le passage à un modèle par ticker modifie implicitement la nature du problème : on ne cherche plus à apprendre une relation “moyenne” valable pour l’ensemble du marché, mais une relation locale, potentiellement plus simple et plus stable dans le temps. Dans ce cadre, un R² de l’ordre de 0.025 reste faible en valeur absolue, mais il est non trivial pour des données de rendements financiers et comparable, voire supérieur, aux ordres de grandeur rapportés dans la littérature pour des modèles de prédiction de rendements.

Enfin, il est important de souligner que ces R² plus élevés ne signifient pas nécessairement une exploitabilité économique immédiate. Un modèle par ticker peut capter des régularités spécifiques mais aussi être plus sensible au sur-ajustement et moins robuste hors-échantillon long terme. Néanmoins, ces résultats suggèrent que la prédiction des rendements gagne à être abordée de manière plus désagrégée, soit via des modèles spécifiques par titre, soit via des modèles hiérarchiques ou multi-modèles capables de tenir compte de l’hétérogénéité entre actions.


# Conslusion

Dans l’article de Gu, Kelly & Xiu (2020), Empirical Asset Pricing via Machine Learning, les auteurs cherchent explicitement à prédire les rendements excédentaires mensuels des actions à partir d’un grand nombre de caractéristiques. Leur protocole expérimental est construit de façon à mimer la situation d’un investisseur réel :

les modèles sont entraînés sur une première partie de l’historique, puis évalués hors-échantillon sur une plage de dates ultérieure, distincte et plus récente, éventuellement dans un cadre de type walk-forward (les paramètres peuvent être ré-estimés au fil du temps, mais le test reste sur un bloc “futur” par rapport au train).

Autrement dit, chez Gu, Kelly & Xiu, le jeu de test correspond à une période temporelle séparée, sur laquelle on n’utilise que l’information disponible dans le passé pour prédire les rendements du futur. C’est exactement la logique opérationnelle d’un fonds : on veut savoir comment le modèle se comporte quand on avance dans le temps, sans réutiliser, même indirectement, des informations futures dans la phase d’entraînement.

Ici, Nous avons adopté une approche différente, davantage pensée comme un exercice de prédiction statistique i.i.d que comme un backtest de trading. Concrètement, on part d’un panel (Ticker,Date) et on construit, pour chaque ligne, une cible définie comme le rendement moyen futur sur \(H\) jours à partir des log-rendements :

$$
\frac{1}{H}\sum_{k=1}^{H} r_{t+k}
$$

pour une date \(t\).
.

Afin de limiter les fuites d’information les plus immédiates, nous ne conservons ensuite qu’une date sur H+1 dans chaque série temporelle (thinning) : cela garantit qu’aucun jour utilisé dans la fenêtre “futur” d’une observation ne réapparaisse comme date d’observation pour une autre.

Sur cet ensemble de dates espacées — toujours triées chronologiquement — nous réalisons ensuite un split 60 % / 20 % / 20 % en train / validation / test, en prenant successivement les premières dates (train), les suivantes (validation), puis les dernières (test).

Chaque sous-échantillon correspond donc à un bloc temporel sur ces dates retenues, mais, comme une grande partie des jours est volontairement écartée, le test ne représente plus une plage dense et continue d’observations (comme dans un backtest financier), mais plutôt un sous-échantillon éclairci de l’historique, sur lequel nous évaluons la capacité du modèle (XGBoost et autres modèles simples) à capturer la relation entre caractéristiques et rendements moyens futurs.

Dans ce cadre, nous obtenons des R² hors-échantillon sensiblement plus élevés que ceux rapportés par Gu, Kelly & Xiu (typiquement autour de 2.5 % dans notre cas pour certains modèles spécialisés sur un ticker, contre des ordres de grandeur de 1–2 % chez eux avec des modèles beaucoup plus sophistiqués). Ces résultats montrent que, d’un point de vue purement prédictif, et malgré l’utilisation de modèles relativement simples, notre protocole de split i.i.d. modifié permet effectivement d’obtenir de bonnes performances en termes de MSE / R².

Il convient toutefois de remettre ces ordres de grandeur en perspective. Pris isolément, un R² de l’ordre de 2.5 % peut sembler très faible au regard des standards usuels de la régression appliquée à des phénomènes physiques ou économiques plus stables. Cependant, dans le contexte de la prédiction des rendements financiers, une telle valeur constitue au contraire un résultat notable.

Les rendements d’actifs financiers sont caractérisés par un rapport signal–bruit extrêmement faible, une forte composante aléatoire et une instabilité temporelle marquée des relations entre variables explicatives et variable cible. Une grande partie de la variance des rendements est due à des chocs imprévisibles (annonces macroéconomiques, événements géopolitiques, flux de liquidité, réactions comportementales), ce qui limite structurellement le pouvoir explicatif de tout modèle statistique. Dans ce contexte, un R² hors-échantillon positif, même de quelques pourcents, indique déjà que le modèle parvient à extraire un signal prédictif réel au-delà du bruit.

Ce point est largement documenté dans la littérature empirique en finance, où de nombreux travaux de référence rapportent des R² hors-échantillon proches de zéro, voire négatifs, pour la prédiction des rendements, y compris à l’aide de modèles sophistiqués. Ainsi, des R² de l’ordre de 1–2 % sont souvent considérés comme substantiels lorsqu’ils sont obtenus de manière robuste et sans fuite d’information. Dans cette perspective, les valeurs observées ici, autour de 2.5 % pour certains modèles spécialisés par ticker, se situent dans la partie haute de ce qui est empiriquement atteignable et témoignent d’une capacité prédictive non triviale.

En revanche, il est important de souligner que ces bonnes performances ne sont pas directement exploitables en production par un fonds : un investisseur doit prendre des décisions sur des plages strictement futures par rapport aux données d’entraînement. Pour évaluer une stratégie de manière réaliste, il est donc plus pertinent d’utiliser un schéma de type Gu, Kelly & Xiu, avec un bloc de test séparé dans le futur, voire un protocole de prévision walk-forward sur les dernières années. Dans un tel cadre, nos R² seraient très probablement plus faibles, mais l’évaluation serait beaucoup plus proche de la réalité du métier (prévoir des rendements à venir, pas ré-échantillonner tout l’historique comme s’il était i.i.d.).

En résumé, notre protocole de split améliore les performances de prédiction en niveau (R²), mais au prix d’un éloignement par rapport au cadre “production” d’un fonds. Le découpage en bloc temporel futur, tel qu’utilisé par Gu, Kelly & Xiu, est moins flatteur en termes de R² mais beaucoup plus pertinent dès lors que l’objectif est réellement de déployer une stratégie d’investissement.